In [ ]:
import os
import torch
import datasets
from transformers import (
    LlavaNextForConditionalGeneration,
    LlavaNextProcessor
)
hf_token = os.environ['HF_TOKEN']

In [ ]:
model = LlavaNextForConditionalGeneration.from_pretrained(
    'ShinDJ/llava-next-docviz',
    attn_implementation='flash_attention_2',
    torch_dtype=torch.bfloat16,
    token=hf_token
).to('cuda:0')

processor = LlavaNextProcessor.from_pretrained('ShinDJ/llava-next-docviz',token=hf_token)
test_data = datasets.load_dataset('MLP-VLM/bllossom-vision','aihub-docviz',split='test')

In [ ]:
idx = 303
image = test_data[idx]['images']
message = [
    {'role':'user','content': '<image>\n'+test_data[idx]['instruction']}
]
response = test_data[idx]['response']
chat_message = processor.tokenizer.apply_chat_template(message,tokenize=False,add_generation_prompt=True)

inputs = processor(
    text=chat_message,
    images=image,
    add_special_tokens=False,
    return_tensors='pt'
).to(model.device)

output = model.generate(**inputs,max_new_tokens=512)
output_texts = processor.tokenizer.decode(output[0],eos_token_id=processor.tokenizer.convert_tokens_to_ids('<|eot_id|>'))

start_idx = output_texts.find('assistant<|end_header_id|>\n\n') + len('assistant<|end_header_id|>\n\n')
image.show()
print("Predict :: ",output_texts[start_idx:].replace('<|eot_id|>',''))
print("Label :: ",response)